In [9]:
import pandas as pd
merged_df = pd.read_csv('data/merged_df.csv')

import pymongo
from pymongo import MongoClient

# # Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')  # MongoDB connection string

In [12]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import model_from_json

# Load the dataset (assuming merged_df is already loaded)
# merged_df = pd.read_csv('path_to_wer_merged_df.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

dropdown_options = ['TSLA', 'AAPL', 'GOOG', 'AMZN', 'MSFT']

app.layout = html.Div([
    html.H1("Stock Price Prediction Dashboard"),
    dcc.Dropdown(
        id='stock-dropdown',
        options=dropdown_options,
        value='AAPL',  # Default value
        placeholder="Select a stock"
    ),
    html.Div([
        html.Label("Sequence Length:"),
        dcc.Input(id='seq-length-input', type='number', value=7),
    ]),
    html.Button('Predict', id='predict-button', n_clicks=0),
    html.Div(id='prediction-output'),
    dcc.Graph(id='prediction-graph')
])

@app.callback(
    [Output('prediction-output', 'children'),
     Output('prediction-graph', 'figure')],

    [Input('predict-button', 'n_clicks')],
    [Input('stock-dropdown', 'value'),
     Input('seq-length-input', 'value')]
)
def update_prediction(n_clicks, ticker, seq_length):
    if n_clicks == 0:
        return "", {}

    # Load the pre-trained model
    model = load_model(f'models/rnn/{ticker}_{seq_length}.keras')

    ticker_df = merged_df[merged_df['ticker'] == ticker]

    data, labels = prep_ticker_data(ticker, merged_df)
    X_train, X_test, y_train, y_test, scaler, split = scale_and_split(labels, seq_length)

    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)

    return "", {}

    # Make predictions
    # predictions = model.predict(['2022-10-13'])

    # # Create the Plotly figure
    # fig = go.Figure()

    # # Add the actual closing price trace
    # fig.add_trace(go.Scatter(
    #     x=ticker_df['date'][seq_length:],
    #     y=ticker_df['close'][seq_length:],
    #     mode='lines',
    #     name='Actual Closing Price',
    #     line=dict(color='blue')
    # ))

    # # Add the predicted closing price trace
    # fig.add_trace(go.Scatter(
    #     x=ticker_df['date'][seq_length:],
    #     y=predictions.flatten(),
    #     mode='lines',
    #     name='Predicted Closing Price',
    #     line=dict(color='red')
    # ))

    # # Update the lawet
    # fig.update_lawet(
    #     title=f'{ticker} Closing Price Prediction',
    #     xaxis_title='Date',
    #     yaxis_title='Closing Price',
    #     legend=dict(x=0, y=1)
    # )

    # return f"Predicted closing price for {ticker}: {predictions[-1][0]:.2f}", fig

if __name__ == '__main__':
    app.run_server(debug=True)